# Question - Answering with Retrieval

본 대회의 과제는 중앙정부 재정 정보에 대한 **검색 기능**을 개선하고 활용도를 높이는 질의응답 알고리즘을 개발하는 것입니다. <br>이를 통해 방대한 재정 데이터를 일반 국민과 전문가 모두가 쉽게 접근하고 활용할 수 있도록 하는 것이 목표입니다. <br><br>
베이스라인에서는 평가 데이터셋만을 활용하여 source pdf 마다 Vector DB를 구축한 뒤 langchain 라이브러리와 llama-2-ko-7b 모델을 사용하여 RAG 프로세스를 통해 추론하는 과정을 담고 있습니다. <br>( train_set을 활용한 훈련 과정은 포함하지 않으며, test_set  에 대한 추론만 진행합니다. )

# Download Library

In [ ]:
"""
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install transformers[torch] -U

!pip install datasets
!pip install langchain
!pip install langchain_community
!pip install PyMuPDF
!pip install sentence-transformers
!pip install faiss-gpu
"""

# Import Library

In [1]:
import os
import unicodedata

import torch
import pandas as pd
from tqdm import tqdm
import fitz  # PyMuPDF
import pickle

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig,
    TrainingArguments,
    TrainerCallback
)
from accelerate import Accelerator

from trl import SFTConfig, SFTTrainer
from datasets import load_dataset, Dataset
import pickle
import wandb

# Langchain 관련
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate 
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from peft import LoraConfig, get_peft_model

c:\Users\Seo\anaconda3\envs\Search_Baseline\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Vector DB

In [2]:
def process_pdf(file_path, chunk_size=800, chunk_overlap=50):
    """PDF 텍스트 추출 후 chunk 단위로 나누기"""
    # PDF 파일 열기
    doc = fitz.open(file_path)
    text = ''
    # 모든 페이지의 텍스트 추출
    for page in doc:
        text += page.get_text()
    # 텍스트를 chunk로 분할
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunk_temp = splitter.split_text(text)
    # Document 객체 리스트 생성
    chunks = [Document(page_content=t) for t in chunk_temp]
    return chunks


def create_vector_db(chunks, model_path="intfloat/multilingual-e5-base"):
    """FAISS DB 생성"""
    # 임베딩 모델 설정
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True}
    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    # FAISS DB 생성 및 반환
    db = FAISS.from_documents(chunks, embedding=embeddings)
    return db

def normalize_path(path):
    """경로 유니코드 정규화"""
    return unicodedata.normalize('NFC', path)


def process_pdfs_from_dataframe(df, base_directory):
    """딕셔너리에 pdf명을 키로해서 DB, retriever 저장"""
    pdf_databases = {}
    unique_paths = df['Source_path'].unique()
    
    for path in tqdm(unique_paths, desc="Processing PDFs"):
        # 경로 정규화 및 절대 경로 생성
        normalized_path = normalize_path(path)
        full_path = os.path.normpath(os.path.join(base_directory, normalized_path.lstrip('./'))) if not os.path.isabs(normalized_path) else normalized_path
        
        pdf_title = os.path.splitext(os.path.basename(full_path))[0]
        print(f"Processing {pdf_title}...")
        
        # PDF 처리 및 벡터 DB 생성
        chunks = process_pdf(full_path)
        db = create_vector_db(chunks)
        
        # Retriever 생성
        retriever = db.as_retriever(search_type="mmr", 
                                    search_kwargs={'k': 3, 'fetch_k': 8})
        
        # 결과 저장
        pdf_databases[pdf_title] = {
                'db': db,
                'retriever': retriever
        }
    return pdf_databases


# DB 생성

In [3]:
base_directory = './' # Your Base Directory
df = pd.read_csv('./test.csv')
pdf_databases = process_pdfs_from_dataframe(df, base_directory)
pickle_file_path = os.path.join(base_directory, 'pdf_databases_e5_base.pickle')
with open(pickle_file_path, 'wb') as f:
    pickle.dump(pdf_databases, f)

Processing PDFs:   0%|          | 0/9 [00:00<?, ?it/s]

Processing 중소벤처기업부_혁신창업사업화자금(융자)...


c:\Users\Seo\anaconda3\envs\Search_Baseline\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\Seo\anaconda3\envs\Search_Baseline\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Seo\.cache\huggingface\hub\models--intfloat--multilingual-e5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variabl

Processing 보건복지부_부모급여(영아수당) 지원...


Processing PDFs:  22%|██▏       | 2/9 [00:30<01:33, 13.32s/it]

...

Processing PDFs:  78%|███████▊  | 7/9 [00:49<00:09,  4.68s/it]

Processing 「FIS 이슈&포커스」 22-2호 《재정성과관리제도》...


Processing PDFs:  89%|████████▉ | 8/9 [00:53<00:04,  4.51s/it]

Processing 「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...


Processing PDFs: 100%|██████████| 9/9 [00:57<00:00,  6.37s/it]


In [2]:
base_directory = './' # Your Base Directory
df = pd.read_csv('./test.csv')
with open('pdf_databases_cpu.pickle', 'rb') as f:
    pdf_databases = pickle.load(f)

c:\Users\Seo\anaconda3\envs\Search_Baseline\lib\site-packages\torch\storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [ ]:
wandb.init(project="search_competition", entity="tjwjddn980117")

# MODEL Import

In [ ]:
import pandas as pd
import unicodedata
from tqdm import tqdm
from transformers import TrainingArguments, AutoTokenizer, AutoModelForCausalLM, pipeline
from datasets import Dataset, DatasetDict
from trl import SFTTrainer
from peft import LoraConfig
import wandb

# Initialize 

# CSV 파일 읽기
train_df = pd.read_csv('stf_e5_base_train.csv')
eval_df = pd.read_csv('stf_e5_base_eval.csv')

# 포맷팅 함수 정의
def formatting_prompts_func(example):
    input_texts = []
    target_texts = []
    for i in range(len(example)):
        input_text = f"""다음 정보를 바탕으로 질문에 답하세요:
{example['Context'][i]}
질문: {example['Question'][i]}
답변: 
"""
        target_text = example['Answer'][i]
        input_texts.append(input_text)
        target_texts.append(target_text)
    return input_texts, target_texts

# 포맷팅된 텍스트를 데이터셋에 추가
train_inputs, train_targets = formatting_prompts_func(train_df)
eval_inputs, eval_targets = formatting_prompts_func(eval_df)

# 데이터셋을 DataFrame으로 변환하여 쉽게 사용할 수 있게 함
train_df = pd.DataFrame({'input_text': train_inputs, 'target_text': train_targets})
eval_df = pd.DataFrame({'input_text': eval_inputs, 'target_text': eval_targets})

# DataFrame의 일부 데이터 출력
print("Train DataFrame example:")
print(train_df.head())

print("\nEval DataFrame example:")
print(eval_df.head())

In [4]:
from peft import TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING

model_type = "rtzr/ko-gemma-2-9b-it"
default_target_modules = TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING.get(model_type, None)

print(f"Default target modules for {model_type}: {default_target_modules}")

ImportError: cannot import name 'TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING' from 'peft' (c:\Users\Seo\anaconda3\envs\Search_Baseline\lib\site-packages\peft\__init__.py)

In [2]:
def setup_llm_SFTTrainer():
    # 4비트 양자화 설정
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    # 모델 ID 
    #model_id = "beomi/llama-2-ko-7b"
    model_id = "rtzr/ko-gemma-2-9b-it"

    # 토크나이저 로드 및 설정
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.use_default_system_prompt = False

    # 모델 로드 및 양자화 설정 적용
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True
    )

    
    # Load LoRA configuration
    peft_config = LoraConfig(
        r=16,
        lora_alpha=32,
        lora_dropout=0.05,
        bias="none",
        target_modules=[
        "model.embed_tokens.weight", 
        "model.layers.*.input_layernorm.weight", 
        "model.layers.*.post_attention_layernorm.weight", 
        "model.layers.*.pre_feedforward_layernorm.weight", 
        "model.layers.*.post_feedforward_layernorm.weight", 
        "model.norm.weight"
        ],
        task_type="CAUSAL_LM",
    )        
    #for name, param in model.named_parameters():
    #    print(name, param.requires_grad)
    
    training_args = TrainingArguments(
        output_dir="./results",
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        num_train_epochs=15,
        logging_dir='./logs',
        logging_steps=1000,
        save_steps=1000,
        evaluation_strategy="steps"
    )

    #dataset
    train_dataset = load_dataset('csv', data_files='stf_e5_base_train.csv')['train']
    eval_dataset = load_dataset('csv', data_files='stf_e5_base_eval.csv')['train']  
    
#    def formatting_prompts_func(example):
#        output_texts = []
#         for i in range(len(example)):
#             text = template = """다음 정보를 바탕으로 질문에 답하세요. 답변은 꼭 문장으로 하세요. 주어를 꼭 적으세요. :
# {example[Context]}
# 
# 질문: {example[Question]}
# 
# 답변: {example[Answer]}
# """
#             output_texts.append(text)
#         return output_texts
    def formatting_prompts_func(example):
        output_texts = []
        for i in range(len(example)):
            text = template = """{example[Answer]}"""
            output_texts.append(text)
        return output_texts
    trainer = SFTTrainer(
        model=model,
        args=training_args,
        peft_config = peft_config,
        formatting_func=formatting_prompts_func,
        train_dataset = train_dataset,
        eval_dataset = eval_dataset,   
    )

    # Train model
    trainer.train()

    finetuned_model = "gemma_ko_7b_ver2.0"
    # Save trained model
    trainer.model.save_pretrained(finetuned_model)
    
    text_generation_pipeline = pipeline(
        model=trainer.model,
        tokenizer=tokenizer,
        task="text-generation",
        temperature=0.2,
        return_full_text=False,
        max_new_tokens=128, 
    )

    hf = HuggingFacePipeline(pipeline=text_generation_pipeline)
    return hf
    # return text_generation_pipeline

In [3]:
llm = setup_llm_SFTTrainer()

Loading checkpoint shards: 100%|██████████| 10/10 [00:07<00:00,  1.34it/s]


model.embed_tokens.weight True
model.layers.0.self_attn.q_proj.weight False
model.layers.0.self_attn.k_proj.weight False
model.layers.0.self_attn.v_proj.weight False
model.layers.0.self_attn.o_proj.weight False
model.layers.0.mlp.gate_proj.weight False
model.layers.0.mlp.up_proj.weight False
model.layers.0.mlp.down_proj.weight False
model.layers.0.input_layernorm.weight True
model.layers.0.post_attention_layernorm.weight True
model.layers.0.pre_feedforward_layernorm.weight True
model.layers.0.post_feedforward_layernorm.weight True
model.layers.1.self_attn.q_proj.weight False
model.layers.1.self_attn.k_proj.weight False
model.layers.1.self_attn.v_proj.weight False
model.layers.1.self_attn.o_proj.weight False
model.layers.1.mlp.gate_proj.weight False
model.layers.1.mlp.up_proj.weight False
model.layers.1.mlp.down_proj.weight False
model.layers.1.input_layernorm.weight True
model.layers.1.post_attention_layernorm.weight True
model.layers.1.pre_feedforward_layernorm.weight True
model.layer

c:\Users\Seo\anaconda3\envs\Search_Baseline\lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


ValueError: Please specify `target_modules` in `peft_config`

# Langchain 을 이용한 추론

In [7]:
def normalize_string(s):
    """유니코드 정규화"""
    return unicodedata.normalize('NFC', s)

def format_docs(docs):
    """검색된 문서들을 하나의 문자열로 포맷팅"""
    context = ""
    for doc in docs:
        context += doc.page_content
        context += '\n'
    return context

# 결과를 저장할 리스트 초기화
results = []

# DataFrame의 각 행에 대해 처리
for _, row in tqdm(df.iterrows(), total=len(df), desc="Answering Questions"):
    # 소스 문자열 정규화
    source = normalize_string(row['Source'])
    question = row['Question']

    # 정규화된 키로 데이터베이스 검색
    normalized_keys = {normalize_string(k): v for k, v in pdf_databases.items()}
    retriever = normalized_keys[source]['retriever']

    # RAG 체인 구성
    template = """
    다음 정보를 바탕으로 질문에 답하세요:
    {context}

    질문: {question}

    답변:
    """
    prompt = PromptTemplate.from_template(template)

    # RAG 체인 정의
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    # 답변 추론
    #print(f"Question: {question}")
    full_response = rag_chain.invoke(question)

    #print(f"Answer: {full_response}\n")

    # 결과 저장
    results.append({
        "Source": row['Source'],
        "Source_path": row['Source_path'],
        "Question": question,
        "Answer": full_response
    })

Answering Questions:   0%|          | 0/98 [00:00<?, ?it/s]c:\Users\Seo\anaconda3\envs\Search_Baseline\lib\site-packages\transformers\generation\configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [4]:

def normalize_string(s):
    """유니코드 정규화"""
    return unicodedata.normalize('NFC', s)

def format_docs(docs):
    """검색된 문서들을 하나의 문자열로 포맷팅"""
    context = ""
    for doc in docs:
        context += doc.page_content
        context += '\n'
    return context

# 결과를 저장할 리스트 초기화
results = []

# CSV 파일 읽기
df = pd.read_csv('stf_e5_base_test.csv')

# DataFrame의 각 행에 대해 처리
for _, row in tqdm(df.iterrows(), total=len(df), desc="Answering Questions"):
    # 소스 문자열 정규화
    context = row['Context']
    question = row['Question']

    # RAG 체인 구성
    template = """다음 정보를 바탕으로 질문에 답하세요. 답변은 꼭 문장으로 하세요. 주어를 꼭 적으세요. :
    {context}

    질문: {question}

    답변:
    """
    prompt = PromptTemplate.from_template(template)

    # RAG 체인 정의
    rag_chain = ( 
        prompt
        | llm
        | StrOutputParser()
    )

    # 답변 추론
    print(f"Question: {question}")
    full_response = rag_chain.invoke({"context":context, "question":question})

    print(f"Answer: {full_response}\n")
    
    # 결과 저장
    results.append({
        'Question': question,
        'Context': context,
        'Answer': full_response
    })


Answering Questions:   0%|          | 0/98 [00:00<?, ?it/s]c:\Users\Seo\anaconda3\envs\Search_Baseline\lib\site-packages\transformers\generation\configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Question: 2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?


Answering Questions:   1%|          | 1/98 [00:09<14:36,  9.04s/it]

Answer: 질문: 2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?
    답변: 2022년 혁신창업사업화자금(융자)의 예산은 2,300,000백만원입니다.
    질문: 2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?
    답변: 2022년 혁신창업

Question: 중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?


c:\Users\Seo\anaconda3\envs\Search_Baseline\lib\site-packages\transformers\generation\configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Answering Questions:   2%|▏         | 2/98 [00:17<14:14,  8.90s/it]

...

Answer: 2019년입니다.

    질문: 청년전용창업자금이 혁신창업지원자금에 통합된 연도는 언제인가요?

    답변:
    2020년입니다.

    질문: 미래기술육성자금과 고성장촉진자금이 폐지된 연도는 언제인가요?

    답변:
    2022년입니다.

    질문: 성장공유형 대출방식이 추가된 연도는 언제

Question: 창업기반지원과 신청 대상이 중복인 자금이 어떤 것이며, 이 자금이 폐지된 연도는 언제인가요?


c:\Users\Seo\anaconda3\envs\Search_Baseline\lib\site-packages\transformers\generation\configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Answering Questions:   7%|▋         | 7/98 [00:56<12:05,  7.97s/it]

Answer: 창업기반지원과 신청 대상이 중복인 자금이 어떤 것이며, 이 자금이 폐지된 연도는 언제인가요?
    창업기반지원과 신청 대상이 중복인 자금이 어떤 것이며, 이 자금이 폐지된 연도는 언제인가요?
    창업기반지원과 신청 대상이 중복인 자금이 어떤 것이며, 이 자금이 폐지된 연도는 언제인가요?
    창업

Question: 혁신창업사업화자금(융자) 사업을 시행하는 주체는 누구인가요?


c:\Users\Seo\anaconda3\envs\Search_Baseline\lib\site-packages\transformers\generation\configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Answering Questions:   7%|▋         | 7/98 [01:01<13:21,  8.81s/it]


KeyboardInterrupt: 

# Submission

In [9]:
# 제출용 샘플 파일 로드
submit_df = pd.read_csv("./sample_submission.csv")

# 생성된 답변을 제출 DataFrame에 추가
submit_df['Answer'] = [item['Answer'] for item in results]
submit_df['Answer'] = submit_df['Answer'].fillna("데이콘")     # 모델에서 빈 값 (NaN) 생성 시 채점에 오류가 날 수 있음 [ 주의 ]

# 결과를 CSV 파일로 저장
submit_df.to_csv("./gemma_7b_ver2.0_submission.csv", encoding='UTF-8-sig', index=False)